In [ ]:
"""
Model
"""
import os
import csv
import math
import numpy as np
import cv2
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Input, Activation, Dropout, Conv2D, Flatten, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
LABELS_FILENAME = 'data/driving_log.csv'

def genTrainingData(data):
    while True:
        src = genNormalizedData(genAugmentedViews(data))
        for sample in src:
            yield sample

def enumDrivingLog(driving_log_fname):
    with open(driving_log_fname, 'r') as labels_file:
        rdr=csv.DictReader(labels_file)
        for row in rdr:
            yield row

def filterDrivingLog(src, threshold, prob):
    for row in src:
        steering = float(row['steering'])
        if abs(steering) > threshold or np.random.randint(0,100) <= prob:
            yield row

def genAugmentedViews(src):
    for row in src:
        steering = float(row['steering'])
        center = readImage(row['center'])
        left = readImage(row['left'])
        right = readImage(row['right'])
        
        # adjust steering for left and right cameras
        left_steer = steering + 0.25
        right_steer = steering - 0.25
        #print('center %.2f, left: %.2f, right: %.2f' % (steering, left_steer, right_steer) )
    
        yield (center, steering)
        yield (left, left_steer)
        yield (right, right_steer)
        
        yield (addShadow(center), steering)
        yield (addShadow(left), left_steer)
        yield (addShadow(right), right_steer)
        
        yield (cv2.flip(center, 1), steering * -1.)
        
        yield (adjustBrightness(center, 2), steering)
        yield (adjustBrightness(center, 3), steering)
        yield (adjustBrightness(left, 2), left_steer)
        yield (adjustBrightness(left, 3), left_steer)
        yield (adjustBrightness(right, 2), right_steer)
        yield (adjustBrightness(right, 3), right_steer)
    
def addShadow(img):
    y,u,v = cv2.split(cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb))
    y = y.astype(np.int32)
    mask = np.zeros(y.shape, dtype=np.int32)
    
    x1 = np.random.uniform() * y.shape[1]
    x2 = np.random.uniform() * y.shape[1]
    slope = float(y.shape[0]) / (x2 - x1)
    intercept = -(slope * x1)
    for j in range(mask.shape[0]):
        for i in range(mask.shape[1]):
            if j > (i*slope)+intercept:
                mask[j,i] -= 80

    y += mask
    y = np.clip(y, 0,255).astype(np.uint8)
    return cv2.cvtColor(cv2.merge((y,u,v)), cv2.COLOR_YCrCb2RGB)
                
def adjustBrightness(img, d):
    h,s,v = cv2.split(cv2.cvtColor(img, cv2.COLOR_RGB2HSV))
    return cv2.cvtColor(cv2.merge((h,s,v // d)), cv2.COLOR_HSV2RGB)
        
def genNormalizedData(src):
    for img,steering in src:
        normImg = normalizeImage(convColorSpace(img))
        normSteering = round(steering,2) / 2.
        #print('steer: %.3f, normSteer: %.3f' % (steering, normSteering ))
        croppedImg = cropImage(normImg)
        x = croppedImg.reshape(1,*croppedImg.shape)
        y = np.array(normSteering, ndmin=1, dtype=np.float)
        yield x,y
        
"""
Operations done for every image - both training and predictions
"""

def readImage(img_fname):
    img_path = os.path.join('data',img_fname.strip())
    img = cv2.imread(img_path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def convColorSpace(img):
    y,u,v = cv2.split(cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb))
    return cv2.merge((y,u,v))
    #return img

def cropImage(img):
    ht = img.shape[0]
    cropHt = ht // 5
    return img[cropHt:ht-cropHt, :]

def normalizeImage(img):
    nImg = ((img / 255.0) - 0.5).astype(np.float32)
    return nImg

def imgForDisplay(img):
    img = img.reshape(img.shape[1:4])
    img = ((img + 0.5) * 255).astype(np.uint8)
    img = cv2.cvtColor(img, cv2.COLOR_YCrCb2RGB)
    return img

data = list(filterDrivingLog(enumDrivingLog(LABELS_FILENAME), 0.01, 25))

# split into train and validation
#train, test = train_test_split(data, test_size=0.25)
#train = shuffle(train)
train = data

imageGenerator = genNormalizedData(genAugmentedViews([train[0]]))
imgListExample = list(imageGenerator)
numImagesPerSample = len(imgListExample)
numRows = len(train) * numImagesPerSample
exampleImg = imgForDisplay(imgListExample[0][0])
image_shape = exampleImg.shape
print("NumSamples: {0}, Shape:{1}".format(numRows, image_shape))

for normImg,normSteering in imgListExample:
    plt.figure()
    plt.axis('off')
    img = imgForDisplay(normImg)
    ht = img.shape[0]+10
    steering = normSteering * 2.
    plt.imshow(img)
    plt.text(0,ht,'Steering: %.3f' % steering)


In [ ]:
MODEL_FILE = 'data/model.h5'
if os.path.exists(MODEL_FILE):
    print("Loading from file")
    model = load_model(MODEL_FILE)
else:
    print("Creating model")
    model = Sequential()

    model.add(Conv2D(8, 5, 5, input_shape=image_shape))
    model.add(MaxPooling2D((2,2)))
    model.add((Dropout(0.5)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(16, 5, 5))
    model.add(MaxPooling2D((2,2)))
    model.add((Dropout(0.5)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(64, 5, 5))
    model.add(MaxPooling2D((2,2)))
    model.add((Dropout(0.5)))
    model.add(Activation('relu'))

    model.add(Conv2D(128, 3, 3))
    model.add(MaxPooling2D((2,2)))
    model.add((Dropout(0.5)))
    model.add(Activation('relu'))


    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='softmax'))

model.summary()


In [ ]:
# TODO: Compile and train the model here.
from keras.optimizers import SGD
sgd = SGD(lr=0.0001)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])
history = model.fit_generator(genTrainingData(train), samples_per_epoch=numRows, nb_epoch=5, verbose=1, max_q_size=50)

In [ ]:
model.save(MODEL_FILE)